In [ ]:
!pip install torch transformers datasets pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
import torch
from datasets import load_dataset
import os

In [ ]:
dataset = load_dataset("community-datasets/offenseval_dravidian", "tamil")

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 35139
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 4388
    })
})


In [ ]:
df = pd.DataFrame(dataset['train'])
df = df.dropna(subset=['label'])

In [ ]:
df['label'] = df['label'].replace({2: 1, 3: 1, 4: 1})

In [ ]:
df = df[df['label'] != 5]

In [ ]:
df['label'].value_counts()

label
0    25425
1     8260
Name: count, dtype: int64

In [ ]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame with columns 'text' and 'label'
# Initial class distribution
print(df['label'].value_counts())

# Separate features and target variable
X = df['text'].values.reshape(-1, 1)  # Reshape text column to 2D array
y = df['label']

# Oversample the minority class (label 1)
oversampler = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversampler.fit_resample(X, y)

# Combine the oversampled data back into a DataFrame
df_over = pd.DataFrame(X_over, columns=['text'])
df_over['label'] = y_over

# Display the class distribution after oversampling
print(df_over['label'].value_counts())

# Undersample the majority class (label 0)
undersampler = RandomUnderSampler(sampling_strategy='not minority')
X_under, y_under = undersampler.fit_resample(df_over['text'].values.reshape(-1, 1), df_over['label'])

# Combine the undersampled data back into a DataFrame
df_resampled = pd.DataFrame(X_under, columns=['text'])
df_resampled['label'] = y_under

# Display the final class distribution
print(df_resampled['label'].value_counts())

# Display the first few rows of the resampled DataFrame
print(df_resampled.head())

# Split the resampled data into training and validation sets
train_df, val_df = train_test_split(df_resampled, test_size=0.2, stratify=df_resampled['label'])

# Display the class distribution in the training and validation sets
print(train_df['label'].value_counts())
print(val_df['label'].value_counts())


label
0    25425
1     8260
Name: count, dtype: int64
label
0    25425
1    25425
Name: count, dtype: int64
label
0    25425
1    25425
Name: count, dtype: int64
                                                text  label
0                  movie vara level la Erika poguthu      0
1          Padam nalla comedy padama irukum polaye..      0
2  karthick subburaj anne .... intha padam vetri ...      0
3  கவுண்டர் தேவர்.சார்பாக வெற்றி பெற வாழ்த்துக்கள் 🦁      0
4  ippo intha trailer ah parkuravana oru like pod...      0
label
0    20340
1    20340
Name: count, dtype: int64
label
0    5085
1    5085
Name: count, dtype: int64


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [ ]:


class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }




In [ ]:
MAX_LEN = 128
BATCH_SIZE = 16

import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Number of devices:", torch.cuda.device_count())
    # Set device to the first available GPU
    device = torch.device("cuda:0")
    print("Current device:", device)
else:
    print("CUDA is not available.")
device = torch.device("cuda")
train_dataset = TextClassificationDataset(
    texts=train_df['text'].to_numpy(),
    labels=train_df['label'].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

val_dataset = TextClassificationDataset(
    texts=val_df['text'].to_numpy(),
    labels=val_df['label'].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)


CUDA is available. Number of devices: 1
Current device: cuda:0


In [ ]:
import torch

# Check for available devices
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)

# Move model to device
model.to(device)


Using device: cuda


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss
10,1.804000
20,1.788400
30,1.743600
40,1.679100
50,1.577500
60,1.415900
70,1.197000
80,0.937500
90,0.843500
100,0.764200


TrainOutput(global_step=7629, training_loss=0.2896160543949361, metrics={'train_runtime': 1778.2535, 'train_samples_per_second': 68.629, 'train_steps_per_second': 4.29, 'total_flos': 4041868636385280.0, 'train_loss': 0.2896160543949361, 'epoch': 3.0})

In [ ]:
eval_result = trainer.evaluate()
print(f"Evaluation results: {eval_result}")

Evaluation results: {'eval_loss': 0.3508041501045227, 'eval_runtime': 38.1296, 'eval_samples_per_second': 266.722, 'eval_steps_per_second': 16.68, 'epoch': 3.0}


In [ ]:
def predict(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        _, prediction = torch.max(outputs.logits, dim=1)
    return prediction.item()


In [ ]:
new_text = "sir neega great"
label = predict(new_text)
print(f"Predicted label: {label}")

Predicted label: 0


In [ ]:
new_text = "Comali 9looks teaser la varave Ella"
label = predict(new_text)
print(f"Predicted label: {label}")

Predicted label: 1


In [ ]:
model_path = './tammodel'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./tammodel/tokenizer_config.json',
 './tammodel/special_tokens_map.json',
 './tammodel/vocab.txt',
 './tammodel/added_tokens.json')

In [ ]:
import torch

# Save the model's state_dict to a .pt file
torch.save(model.state_dict(), 'tammodel_state_dict.pt')

# Save the entire model (including the architecture)
torch.save(model, 'tammodel.pt')

# Save the tokenizer
tokenizer.save_pretrained('tammodel_tokenizer')


('tammodel_tokenizer/tokenizer_config.json',
 'tammodel_tokenizer/special_tokens_map.json',
 'tammodel_tokenizer/vocab.txt',
 'tammodel_tokenizer/added_tokens.json')

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the state_dict
#model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)
#model.load_state_dict(torch.load('/content/drive/MyDrive/models/tammodel_state_dict.pt'))

# Alternatively, load the entire model
model = torch.load('/content/sample_data/tammodel.pt')

# Set the model to evaluation mode
model.eval()

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('/content/sample_data/tammodel_tokenizer')


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

import torch
from transformers import DistilBertTokenizer

# Function to make predictions
def predict(texts, model, tokenizer, device, max_len=128):
    model.to(device)
    model.eval()

    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)
    return predictions.cpu()  # Move predictions back to CPU

# Example usage
texts = [
    "sir neega great",
    "Comali 9looks teaser la varave Ella"
]

predictions = predict(texts, model, tokenizer, device)
print("Predictions:", predictions.numpy())



Predictions: [0 1]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
